# Створення додатків для генерації тексту

Ви вже бачили протягом цього курсу, що існують основні концепції, такі як запити, і навіть ціла дисципліна, яка називається "інженерія запитів". Багато інструментів, з якими ви можете взаємодіяти, як-от ChatGPT, Office 365, Microsoft Power Platform тощо, підтримують використання запитів для досягнення мети.

Щоб додати такий досвід у додаток, вам потрібно розуміти такі концепції, як запити, доповнення, та вибрати бібліотеку для роботи. Саме це ви й вивчите в цьому розділі.

## Вступ

У цьому розділі ви:

- Дізнаєтеся про бібліотеку openai та її основні концепції.
- Створите додаток для генерації тексту за допомогою openai.
- Зрозумієте, як використовувати такі концепції, як запит, температура та токени для створення додатку генерації тексту.



## Вправа - створення генератора рецептів

Тепер, коли ми розглянули сценарій, давайте напишемо код, який відповідає продемонстрованому сценарію. Для цього виконайте наступні кроки:

1. Використовуйте існуючий файл як початкову точку
1. Створіть змінну `prompt` і змініть зразок коду, як показано нижче:

In [1]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

prompt = "Покажи мені 5 рецептів для страви з наступними інгредієнтами: курка, картопля та цибуля. Для кожного рецепту перелічи всі використані інгредієнти"

response = client.complete(
    messages=[
        {
            "role": "system",
            "content": "Ти корисний асистент.",
        },
        {
            "role": "user",
            "content": prompt,
        },
    ],
    model=model_name,
    # Опціональні параметри
    temperature=1.,
    max_tokens=3000,
    top_p=1.    
)

print(response.choices[0].message.content)

Ось 5 рецептів з використанням курки, картоплі та цибулі. Для кожного з них я зазначу усі інгредієнти:

---

### 1. **Запечена курка з картоплею і цибулею**
**Інгредієнти:**
- Курка (ціла або шматочки)
- Картопля
- Цибуля
- Оливкова або рослинна олія
- Сіль
- Перець
- Паприка
- Часник (за бажанням)
- Розмарин або сушені трави (за бажанням)

**Приготування:**
Заправте курку олією, сіллю, перцем, паприкою і травами. Наріжте картоплю і цибулю, викладіть разом із куркою у форму для запікання. Готуйте в духовці при температурі 180-200°С близько 40-60 хвилин.

---

### 2. **Курячий суп з картоплею і цибулею**
**Інгредієнти:**
- Курячі шматочки (стегна, філе або крильця)
- Картопля
- Цибуля
- Морква
- Часник
- Лавровий лист
- Сіль
- Перець
- Вода або курячий бульйон
- Зелень (кріп, петрушка)

**Приготування:**
Відваріть курку у каструлі до напівготовності. Додайте нарізану картоплю, цибулю, моркву і часник. Приправте сіллю, перцем і лавровим листом. Варіть суп до готовності овочів, а в кінці 

1. Змінимо код наступним чином:

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

# 1. Безпечне отримання токена
# Використовуємо .get(), щоб програма не "падала" з помилкою KeyError, якщо токена немає
token = os.environ.get("GITHUB_TOKEN")
if not token:
    print("Помилка: Не знайдено GITHUB_TOKEN у змінних оточення!")
    # Можна додати вихід або запит токена через input
else:
    endpoint = "https://models.inference.ai.azure.com"
    model_name = "gpt-4o"

    # Ініціалізація клієнта
    client = ChatCompletionsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token),
    )

    # 2. Обробка вводу (input)
    # Додаємо значення за замовчуванням, якщо користувач просто натисне Enter
    no_recipes = input("Кількість рецептів (за замовчуванням 3): ") or "3"
    ingredients = input("Список інгредієнтів (наприклад, курка, сир): ")

    if not ingredients:
        print("Ви не ввели інгредієнти!")
    else:
        # 3. Формування промпту
        prompt = f"Покажи мені {no_recipes} рецептів страв, використовуючи: {ingredients}. Надай список інгредієнтів для кожного."

        try:
            # 4. Виклик моделі
            # Змінено структуру messages на об'єкти (рекомендовано для azure-ai-inference)
            response = client.complete(
                messages=[
                    {"role": "system", "content": "Ти професійний шеф-кухар, який допомагає готувати вдома."},
                    {"role": "user", "content": prompt},
                ],
                model=model_name,
                temperature=0.8, # Трохи знизив для більш стабільних рецептів
                max_tokens=2000, # Збільшив, бо 5 рецептів можуть не влізти в 1000
                top_p=1.0    
            )

            # 5. Вивід результату
            print("\n--- ВАШІ РЕЦЕПТИ ---")
            print(response.choices[0].message.content)

        except Exception as e:
            print(f"Сталася помилка при зверненні до ШІ: {e}")

## Індивідуальне завдання

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

def run_memory_analysis():
    key = os.environ.get("GITHUB_TOKEN")
    client = ChatCompletionsClient(
        endpoint="https://models.inference.ai.azure.com",
        credential=AzureKeyCredential(key),
    )

    print("--- Система автоматизованої діагностики пам'яті ---")
    arch = input("Вкажіть архітектуру (x86/ARM) [default: x86]: ").lower() or "x86"
    
    # Словник з тестовими даними для демонстрації
    samples = {
        "x86": "EAX: 0x12345678, EBX: 0x87654321, EIP: 0x08048000, ESP: 0x7FFFFFFC",
        "arm": "R0: 0x12345678, R12: 0x7FFFFFFF, SP: 0x7FFFFFFC, PC: 0x08048100"
    }
    
    current_dump = samples.get(arch, samples["x86"])
    print(f"\n[Вхідні дані ({arch})]:\n{current_dump}")

    # Побудова технічного завдання для моделі
    analysis_task = (
        f"Ти — експерт з Reverse Engineering. Проаналізуй дамп ({arch}):\n{current_dump}\n"
        "1. Опиши стан ключових регістрів.\n"
        "2. Знайди потенційні вразливості (Stack Overflow, Null pointer).\n"
        "3. Дай поради щодо виправлення коду."
    )

    res = client.complete(
        messages=[
            {"role": "system", "content": "Ти вузькоспеціалізований ШІ для аналізу низькорівневого коду."},
            {"role": "user", "content": analysis_task},
        ],
        model="gpt-4o",
        temperature=0.7
    )

    print("\n" + "="*30)
    print("ТЕХНІЧНИЙ ВИСНОВОК ШІ:")
    print(res.choices[0].message.content)
    print("="*30)

run_memory_analysis()

--- Система автоматизованої діагностики пам'яті ---
